In [2]:
import os
import re
import pandas as pd
import numpy as np
from IPython.display import Audio, display, HTML
from glob import glob
from tqdm import tqdm
tqdm.pandas()

In [62]:
train_df = pd.read_csv('./data/cv-corpus-19-1.0-2024-09-13/uk/train.tsv', sep='\t+')
test_df = pd.read_csv('./data/cv-corpus-19-1.0-2024-09-13/uk/test.tsv', sep='\t+')
dev_df = pd.read_csv('./data/cv-corpus-19-1.0-2024-09-13/uk/dev.tsv', sep='\t+')

/var/folders/5h/w_s4h9xj4y94bv7nx566t24c0000gn/T/ipykernel_1318/3586215335.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_df = pd.read_csv('./data/cv-corpus-19-1.0-2024-09-13/uk/train.tsv', sep='\t+')
/var/folders/5h/w_s4h9xj4y94bv7nx566t24c0000gn/T/ipykernel_1318/3586215335.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_df = pd.read_csv('./data/cv-corpus-19-1.0-2024-09-13/uk/test.tsv', sep='\t+')
/var/folders/5h/w_s4h9xj4y94bv7nx566t24c0000gn/T/ipykernel_1318/3586215335.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators

In [69]:
MAPPING_ENG_TO_UK = {
    'a': 'а',
    'c': 'с',
    'e': 'е',
    'i': 'і',
    'k': 'к',
    'm': 'м',
    'o': 'о',
    'p': 'р',
    'x': 'х',
    'y': 'у'
}

UKRAINIAN_LETTERS = 'абвгґдеєжзиіїйклмнопрстуфхцчшщьюя'
LATIN_RE = re.compile(r'[a-zA-Z]')
MULTISPACE_RE = re.compile(r'\s+')


def normalise_symbols(sentence):
    for key, value in mapping_eng_to_uk.items():
        sentence = sentence.replace(key, value)
        sentence = sentence.replace(key.upper(), value.upper())
    return sentence


def normalise_sentence(sentence):
    sentence = sentence.lower()

    for char in sentence:
        if char != ' ' and char not in UKRAINIAN_LETTERS:
            sentence = sentence.replace(char, '')

    # remove redundant spaces
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    return sentence


train_df['sentence_cleaned'] = train_df['sentence'].apply(lambda x: normalise_symbols(x))
train_df = train_df[~train_df['sentence_cleaned'].apply(lambda x: bool(LATIN_RE.search(x)))]
train_df['sentence_cleaned'] = train_df['sentence_cleaned'].apply(lambda x: normalise_sentence(x))

dev_df['sentence_cleaned'] = dev_df['sentence'].apply(lambda x: normalise_symbols(x))
dev_df = dev_df[~dev_df['sentence_cleaned'].apply(lambda x: bool(LATIN_RE.search(x)))]
dev_df['sentence_cleaned'] = dev_df['sentence_cleaned'].apply(lambda x: normalise_sentence(x))

test_df['sentence_cleaned'] = test_df['sentence'].apply(lambda x: normalise_symbols(x))
test_df = test_df[~test_df['sentence_cleaned'].apply(lambda x: bool(LATIN_RE.search(x)))]
test_df['sentence_cleaned'] = test_df['sentence_cleaned'].apply(lambda x: normalise_sentence(x))

In [71]:
from ukrainian_word_stress import OnAmbiguity, Stressifier
stressify = Stressifier(stress_symbol="+", on_ambiguity=OnAmbiguity.First)

train_df['sentence_stressed_lang_uk'] = train_df['sentence_cleaned'].progress_apply(lambda x: stressify(x))
dev_df['sentence_stressed_lang_uk'] = train_df['sentence_cleaned'].progress_apply(lambda x: stressify(x))
test_df['sentence_stressed_lang_uk'] = train_df['sentence_cleaned'].progress_apply(lambda x: stressify(x))

100%|██████████| 25722/25722 [27:20<00:00, 15.68it/s]


In [ ]:
UKRAINIAN_VOWELS = 'аеєиіїоуюя'
lang_uk_common_errors = {
    "м+ене": "мен+е",
    "с+ебе": "себ+е",
    "й+ого": "йог+о",
    "сер+ед": "с+еред",
    "вод+у": "в+оду",
    "х+отів": "хот+ів"
}


def correct_stress_marks(sentence):
    words = sentence.split()
    new_words = []

    for word in words:
        if word in lang_uk_common_errors:
            word = lang_uk_common_errors[word]

        if word.count('+') > 1:
            parts = word.split('+')
            word = parts[0] + '+' + ''.join(parts[1:])

        num_vowels = sum(int(char in UKRAINIAN_VOWELS) for char in word)

        if '+' not in word and num_vowels==1:
            for i, char in enumerate(word):
                if char in UKRAINIAN_VOWELS:
                    word = word[:i+1] + '+' + word[i+1:]
                    break
        new_words.append(word)

    return ' '.join(new_words)

train_df['sentence_stressed_corrected_lang_uk'] = train_df['sentence_stressed_lang_uk'].progress_apply(lambda x: correct_stress_marks(x))
dev_df['sentence_stressed_corrected_lang_uk'] = dev_df['sentence_stressed_lang_uk'].progress_apply(lambda x: correct_stress_marks(x))
test_df['sentence_stressed_corrected_lang_uk'] = test_df['sentence_stressed_lang_uk'].progress_apply(lambda x: correct_stress_marks(x))

In [ ]:
from accentor_model.predict_word_stress import Stressifier
from huggingface_hub import hf_hub_download

path_to_nemo_model = "../accentor_model/byt5_g2p/experiment/checkpoints/T5G2P.nemo"

if not os.path.exists(path_to_nemo_model):
    path_to_nemo_model = hf_hub_download(
        repo_id="mouseyy/stressifier-byt5-g2p-model",
        filename="T5G2P.nemo",
        token="your_token",
    )

stressifier_byt5 = Stressifier()
stressifier_byt5.model = stressifier_byt5.model.to("cuda")

train_df['sentence_stressed_byt5'] = train_df['sentence_cleaned'].progress_apply(lambda x: stressifier_byt5.stressify(x))
dev_df['sentence_stressed_byt5'] = train_df['sentence_cleaned'].progress_apply(lambda x: stressifier_byt5.stressify(x))
test_df['sentence_stressed_byt5'] = train_df['sentence_cleaned'].progress_apply(lambda x: stressifier_byt5.stressify(x))

# Merge lexical stress patterns

In [40]:
UKRAINIAN_VOWELS = 'аеєиіїоуюя'

def correct_stress_marks(sentence):
    words = sentence.split()
    new_words = []

    for word in words:
        num_vowels = sum(int(char in UKRAINIAN_VOWELS) for char in word)
        if num_vowels==1:
            word = word.replace('+', '')
            for i, char in enumerate(word):
                if char in UKRAINIAN_VOWELS:
                    word = word[:i+1] + '+' + word[i+1:]
                    break
        new_words.append(word)

    return ' '.join(new_words)

train_df['sentence_stressed_corrected_lang_uk'] = train_df['sentence_stressed_corrected_lang_uk'].progress_apply(lambda x: correct_stress_marks(x))
dev_df['sentence_stressed_corrected_lang_uk'] = dev_df['sentence_stressed_corrected_lang_uk'].progress_apply(lambda x: correct_stress_marks(x))
test_df['sentence_stressed_corrected_lang_uk'] = test_df['sentence_stressed_corrected_lang_uk'].progress_apply(lambda x: correct_stress_marks(x))

100%|██████████| 10102/10102 [00:00<00:00, 132939.03it/s]


In [41]:
def merge_sentence(row):
    sentence_lang_uk = row.get('sentence_stressed_corrected_lang_uk', '')
    sentence_byt5 = row.get('sentence_stressed_byt5', '')

    words_lang_uk = sentence_lang_uk.split()
    words_byt5 = sentence_byt5.split()

    new_words = []
    for word_dict, word_model in zip(words_lang_uk, words_byt5):
        if '+' in word_dict:
            new_words.append(word_dict)
        else:
            new_words.append(word_model)
    return ' '.join(new_words)

train_df['sentence_merged'] = train_df.apply(merge_sentence, axis=1)
dev_df['sentence_merged'] = dev_df.apply(merge_sentence, axis=1)
test_df['sentence_merged'] = test_df.apply(merge_sentence, axis=1)

In [42]:
def shift_stress_marks_left(text: str, stress_mark="+"):
    """
    Examples:
    >>> shift_stress_marks_left('У+ вишне+вому садку+')
    '+У вишн+евому садк+у'
    >>> shift_stress_marks_left('Приві+т')
    'Прив+іт'
    >>> shift_stress_marks_left('Ти+ біжи+ш')
    'Т+и біж+иш'
    """
    text_list = list(text)

    i = 1
    while i <= len(text_list) - 1:
        if text_list[i] == stress_mark:
            text_list[i - 1], text_list[i] = text_list[i], text_list[i - 1]
            i += 1
        i += 1
    return "".join(text_list)

train_df['sentence_merged'] = train_df['sentence_merged'].apply(shift_stress_marks_left)
dev_df['sentence_merged'] = dev_df['sentence_merged'].apply(shift_stress_marks_left)
test_df['sentence_merged'] = test_df['sentence_merged'].apply(shift_stress_marks_left)

# Add Phonemization

In [43]:
from data.phonemizer import Transcriptor

def phonemize(sentence):
    transcriptor = Transcriptor(sentence)
    return transcriptor.to_ipa()

train_df['sentence_phonemes'] = train_df['sentence_merged'].apply(phonemize)
dev_df['sentence_phonemes'] = dev_df['sentence_merged'].apply(phonemize)
test_df['sentence_phonemes'] = test_df['sentence_merged'].apply(phonemize)

In [52]:
whisper_train_df = pd.read_csv("./data/prepared/new_train.tsv", sep='\t', engine='python', index_col=0)
whisper_train_df = whisper_train_df[['sentence_id', 'whisper_annotation']]

whisper_dev_df = pd.read_csv("./data/prepared/new_dev.tsv", sep='\t', engine='python', index_col=0)
whisper_dev_df = whisper_dev_df[['sentence_id', 'whisper_annotation']]

whisper_test_df = pd.read_csv("./data/prepared/new_test.tsv", sep='\t', engine='python', index_col=0)
whisper_test_df = whisper_test_df[['sentence_id', 'whisper_annotation']]

train_df = pd.merge(train_df, whisper_train_df, on='sentence_id')
dev_df = pd.merge(dev_df, whisper_dev_df, on='sentence_id')
test_df = pd.merge(test_df, whisper_test_df, on='sentence_id')

In [53]:
from jiwer import wer

UKRAINIAN_LETTERS = 'абвгґдеєжзиіїйклмнопрстуфхцчшщьюя'
LATIN_RE = re.compile(r'[a-zA-Z]')
MULTISPACE_RE = re.compile(r'\s+')


def normalise_sentence(sentence):
    sentence = sentence.lower()

    for char in sentence:
        if char != ' ' and char not in UKRAINIAN_LETTERS:
            sentence = sentence.replace(char, '')

    # remove redundant spaces
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    return sentence

train_df['whisper_cleaned'] = train_df['whisper_annotation'].apply(normalise_sentence)
dev_df['whisper_cleaned'] = dev_df['whisper_annotation'].apply(normalise_sentence)
test_df['whisper_cleaned'] = test_df['whisper_annotation'].apply(normalise_sentence)

train_df['wer_score'] = train_df.apply(lambda row: wer(row['sentence_cleaned'], row['whisper_cleaned']), axis=1)
dev_df['wer_score'] = dev_df.apply(lambda row: wer(row['sentence_cleaned'], row['whisper_cleaned']), axis=1)
test_df['wer_score'] = test_df.apply(lambda row: wer(row['sentence_cleaned'], row['whisper_cleaned']), axis=1)

In [60]:
print(train_df[train_df['wer_score'] == 0].shape)
print(train_df[train_df['wer_score'] <= 0.1].shape)
print(train_df[train_df['wer_score'] <= 0.2].shape)

(13705, 22)
(14980, 22)
(18812, 22)


In [62]:
train_df[train_df['wer_score'] <= 0.3]

,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,...,segment,sentence_cleaned,sentence_stressed_lang_uk,sentence_stressed_corrected_lang_uk,sentence_stressed_byt5,sentence_merged,sentence_phonemes,whisper_annotation,whisper_cleaned,wer_score
3,a3428c7fcafc7c12bbef4e320691b3fe9a68395f288d29...,common_voice_uk_23807860.mp3,92cf327169a979ece84c590a24ce022bf0f4392ba61190...,— Що мовити готам?,2,0,twenties,female_feminine,uk,NaN,...,NaN,що мовити готам,що мо+вити го+там,що+ мо+вити го+там,що+ мо+вити го+там,щ+о м+овити г+отам,ʃʧ'o m'ovɪtɪ ɦ'otɑm,Що мовити готам?,що мовити готам,0.000000
4,a3428c7fcafc7c12bbef4e320691b3fe9a68395f288d29...,common_voice_uk_23807907.mp3,0667fb66991ae58858f68379c26296dac694e7ecefeb62...,"— А я-м думала, що ти єси або князь, або хоч б...",2,0,twenties,female_feminine,uk,NaN,...,NaN,а я м думала що ти єси або князь або хоч болярин,а я м ду+мала що ти єси+ або+ князь або+ хоч б...,а+ я+ м ду+мала що+ ти+ єси+ або+ кня+зь або+ ...,а+ я+ м ду+мала що+ ти+ є+си або+ кня+зь або+ ...,+а +я м д+умала щ+о т+и єс+и аб+о кн+язь аб+о ...,'ɑ j'ɑ m d'umɑɫɑ ʃʧ'o t'ɪ jɛs'ɪ ɑb'o knʲ'ɑzʲ ɑ...,"А я б думала, що ти єси або князь, або хоч бу...",а я б думала що ти єси або князь або хоч булярин,0.166667
6,a3428c7fcafc7c12bbef4e320691b3fe9a68395f288d29...,common_voice_uk_23807910.mp3,a51a6cee0ff133cbe7754ecb9d20135762302a353e2830...,— Збиткуєш?!,2,0,twenties,female_feminine,uk,NaN,...,NaN,збиткуєш,збитку+єш,збитку+єш,збитку+єш,збитк+уєш,zbɪtk'ujɛʃ,Збиткуєш?,збиткуєш,0.000000
7,a3428c7fcafc7c12bbef4e320691b3fe9a68395f288d29...,common_voice_uk_23807911.mp3,4961c18c794dac90d25e26fcc90928dc91a600a3824a46...,— Княже!.. Княже Богдане!..,2,0,twenties,female_feminine,uk,NaN,...,NaN,княже княже богдане,кня+же кня+же богдане,кня+же кня+же богдане,кня+же кня+же богда+не,кн+яже кн+яже богд+ане,knʲ'ɑʒɛ knʲ'ɑʒɛ boɦd'ɑnɛ,"Княже, княже Богдане!",княже княже богдане,0.000000
9,a3428c7fcafc7c12bbef4e320691b3fe9a68395f288d29...,common_voice_uk_23807913.mp3,9e90df7ac3eb76e6452f50585b26acd4906c4cc38af7db...,— Хто повідав тобі?,2,0,twenties,female_feminine,uk,NaN,...,NaN,хто повідав тобі,хто повідав тобі+,хто+ повідав тобі+,хто+ повіда+в тобі+,хт+о повід+ав тоб+і,xt'o povʲid'ɑw tobʲ'i,Хто повідав тобі?,хто повідав тобі,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25716,1f2545a0ef34e75af023a4f7518b2f30bbcbd787f6b743...,common_voice_uk_36922750.mp3,0ebb28776e4735712d9afa9480db01a5efc5e54b7e4601...,Тому я підніс його одною рукою і відкинув від ...,2,0,thirties,male_masculine,uk,NaN,...,NaN,тому я підніс його одною рукою і відкинув від ...,тому+ я підні+с йо+го одно+ю руко+ю і відки+ну...,тому+ я+ підні+с йо+го одно+ю руко+ю і+ відки+...,тому+ я+ підні+с його+ одно+ю руко+ю і+ відки+...,том+у +я підн+іс й+ого одн+ою рук+ою +і відк+и...,tom'u j'ɑ pʲidʲnʲ'is j'oɦo odn'oju ruk'oju 'i ...,Тому я підніс його одною рукою і відкинув від...,тому я підніс його одною рукою і відкинув від ...,0.000000
25717,1f2545a0ef34e75af023a4f7518b2f30bbcbd787f6b743...,common_voice_uk_36922751.mp3,0f49cb9f9e8183cde8a8db09bd15e2d685796c81972a68...,Того ж він такий сумний та задуманий завжди.,2,0,thirties,male_masculine,uk,NaN,...,NaN,того ж він такий сумний та задуманий завжди,того+ ж він таки+й сумни+й та заду+маний за+вжди+,того+ ж ві+н таки+й сумни+й та+ заду+маний за+...,того+ ж ві+н таки+й сумни+й та+ заду+маний за+...,тог+о ж в+ін так+ий сумн+ий т+а зад+уманий з+а...,toɦ'o ʒ vʲ'in tɑk'ɪj sumn'ɪj t'ɑ zɑd'umɑnɪj z'...,Того ж він такий сумний та задуманий завжди.,того ж він такий сумний та задуманий завжди,0.000000
25718,1f2545a0ef34e75af023a4f7518b2f30bbcbd787f6b743...,common_voice_uk_36922754.mp3,0f15d53f4a27ed8bd78b1d66a7f47d7c492c5add387985...,Там була робота по душі та були приятелі!,2,0,thirties,male_masculine,uk,NaN,...,NaN,там була робота по душі та були приятелі,там була+ робо+та по душі+ та були+ при+ятелі,та+м була+ робо+та по+ душі+ та+ були+ при+ятелі,та+м бу+ла робо+

In [63]:
train_df_xttsv2_format = train_df[train_df['wer_score'] <= 0.3][['client_id', 'path', 'sentence_phonemes']].copy()
dev_df_xttsv2_format = dev_df[dev_df['wer_score'] <= 0.3][['client_id', 'path', 'sentence_phonemes']].copy()
test_df_xttsv2_format = test_df[test_df['wer_score'] <= 0.3][['client_id', 'path', 'sentence_phonemes']].copy()

columns_mapping = {
    'client_id': 'speaker_name',
    'path': 'audio_file',
    'sentence_phonemes': 'text'
}

train_df_xttsv2_format = train_df_xttsv2_format.rename(columns=columns_mapping)
dev_df_xttsv2_format = dev_df_xttsv2_format.rename(columns=columns_mapping)
test_df_xttsv2_format = test_df_xttsv2_format.rename(columns=columns_mapping)

def change_path(path):
    return "wavs/" + path.split('.')[0] + ".wav"

train_df_xttsv2_format['audio_file'] = train_df_xttsv2_format['audio_file'].apply(change_path)
dev_df_xttsv2_format['audio_file'] = dev_df_xttsv2_format['audio_file'].apply(change_path)
test_df_xttsv2_format['audio_file'] = test_df_xttsv2_format['audio_file'].apply(change_path)

In [64]:
train_df_xttsv2_format.to_csv('./data/prepared/metadata_train.csv', sep='|', index=False)
dev_df_xttsv2_format.to_csv('./data/prepared/metadata_eval.csv', sep='|', index=False)
test_df_xttsv2_format.to_csv('./data/prepared/metadata_test.csv', sep='|', index=False)

# Preparing data for the baseline model

In [31]:
meta_train_df = pd.read_csv('./data/cv-corpus-19-1.0-2024-09-13/metadata_train.csv', sep='|')
meta_test_df = pd.read_csv('./data/cv-corpus-19-1.0-2024-09-13/metadata_test.csv', sep='|')
meta_dev_df = pd.read_csv('./data/cv-corpus-19-1.0-2024-09-13/metadata_eval.csv', sep='|')

merged_train_df = pd.merge(meta_train_df, train_df, on=['audio_file', 'speaker_name'], how='left')
merged_dev_df = pd.merge(meta_dev_df, dev_df, on=['audio_file', 'speaker_name'], how='left')
merged_test_df = pd.merge(meta_test_df, test_df, on=['audio_file', 'speaker_name'], how='left')

merged_train_df['text'] = merged_train_df['text_cleaned']
merged_dev_df['text'] = merged_dev_df['text_cleaned']
merged_test_df['text'] = merged_test_df['text_cleaned']

train_df = train_df[columns_mapping.values()]
test_df = test_df[columns_mapping.values()]
dev_df = dev_df[columns_mapping.values()]

merged_train_df.to_csv('./data/cv-corpus-19-1.0-2024-09-13/metadata_not_norm_train.csv', sep='|', index=False)
merged_dev_df.to_csv('./data/cv-corpus-19-1.0-2024-09-13/metadata_not_norm_eval.csv', sep='|', index=False)
merged_test_df.to_csv('./data/cv-corpus-19-1.0-2024-09-13/metadata_not_norm_test.csv', sep='|', index=False)